In [1]:
import pandas as pd
from os.path import join, isdir

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from icecream import ic
from src.utils import variant_utils as vu
from glob import glob
from itertools import product
from collections import defaultdict
from tqdm.notebook import tqdm
import pickle

In [2]:
indir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/mt_as_clones/variants_init/mt_clones_thresh/"
outdir = "/data/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/mt_as_clones/variants_init/bestparams/"
N_DONORS=2

## Parameters:
# af thresholds
# coverage threshold
# other_af thresholds
# number of cells / fraction cells
# number of other cells / fraction other cells
af_t = 0.1
oth_af_t = 0.1
cov_t = 10
oth_cov_t = 10
ncells = 10
oth_ncells = 0.25 
mean_pos_cov = 0




In [3]:
cells_dir=join(indir, "cells")

In [4]:
def fill_mt_bin(curr_pos, curr_cells):
    print(curr_pos.name)
    curr_pos.loc[curr_cells[curr_pos.name]["other_cells"]] = 0
    curr_pos.loc[curr_cells[curr_pos.name]["clone_cells"]] = 1
    return curr_pos

In [5]:
bin_d = {}
for d in range(N_DONORS): 
    curr_f = join(cells_dir, f"don.{d}_af.{af_t}_othaf.{oth_af_t}_cov.{cov_t}_othcov.{oth_cov_t}.p")
    curr_cells = pickle.load(open(curr_f,'rb'))
    params_results = pd.read_csv(join(indir, f"donor_{d}_thresh_results.tsv"), sep="\t")    


    curr_p = params_results.loc[(params_results["af"]==af_t) &
                       (params_results["oth_af"]==oth_af_t) &
                       (params_results["cov"]==cov_t) &
                       (params_results["oth_cov"]==oth_cov_t) &
                       (params_results["ncells"]==ncells) &
                       (params_results["oth_ncells"]==oth_ncells) &
                       (params_results["mean_cov"]==mean_pos_cov)]
    assert(len(curr_p)==1)
    
    ## Construct a binary cell-by-variant matrix for the kept variants. 
    ## Can add N/A if not in oth cells
    curr_vars = curr_p.iloc[0]["Variants"].split(";")

    all_cells = set()
    for x in curr_cells:
        all_cells = all_cells.union(set(curr_cells[x]["clone_cells"])) 
        all_cells = all_cells.union(set(curr_cells[x]["other_cells"])) 

    mt_bin = pd.DataFrame(index = all_cells, columns=curr_vars)
    bin_d[d] = mt_bin.apply(fill_mt_bin, curr_cells=curr_cells,axis=0)#.fillna(0)


10397G
10463C
10559G
10589A
11251G
11453A
11719A
11761T
11812G
13188T
13368A
14233G
1438G
14674C
146C
14766T
14905A
150T
152C
15326G
15452A
15607G
15928A
16126C
16129A
16294T
16296T
16304C
16355T
16362C
16519C
1888A
1949A
196C
204C
2442C
2623G
263G
2706G
3109C
310C
3244A
3847C
4117C
4216C
4769G
4917G
5147A
5580C
5581G
58C
64T
7028T
709A
73G
750G
7598A
827G
8292A
8461T
8697A
8743A
8860G
930A
9899C
10397G
10463C
10559G
10589A
11251G
11719A
11761T
11812G
13188T
13368A
14233G
1438G
146C
14766T
14905A
150T
15297C
15326G
15452A
15497A
15607G
15928A
16126C
16129A
16294T
16296T
16304C
16355T
16362C
16519C
1888A
1949A
196C
204C
2442C
2623G
263G
2706G
3109C
310C
3244A
3847C
4117C
4216C
4769G
4917G
5147A
5442C
5580C
5581G
58C
64T
7028T
709A
73G
750G
7598A
827G
8292A
8461T
8697A
8860G
930A
9899C


In [6]:
for d in bin_d:
    print(d)
    print(bin_d[d].shape)
    bin_d[d] = bin_d[d].loc[~((bin_d[d])==0).all(axis=1),~((bin_d[d])==0).all(axis=0)]
    print(bin_d[d].shape)
    
    print(bin_d[d].head())
    bin_d[d].fillna(0).to_csv(join(outdir, f"donor_{d}_binary.csv"))
    bin_d[d].to_csv(join(outdir, f"donor_{d}_binary_na.csv"))


bin_comb = pd.concat(list(bin_d.values()),axis=0).fillna(0).astype(int)
bin_comb.to_csv(join(outdir, "combined_binary.csv"))
print("fillna as 0")
bin_comb

0
(9023, 65)
(9023, 65)
                           10397G 10463C 10559G 10589A 11251G 11453A 11719A  \
GCTGTTCAGTAACATG-1_Input        1      0      0      1      0      0      0   
AGCCAGCGTCCGTGCA-1_Flt3l        1    NaN    NaN      1    NaN    NaN    NaN   
TATTGCTTCTTGCGCT-1_Flt3l        1      0      0      1      0      0      0   
CACTAAGTCCCACTAC-1_Control      1      0      0      1      0      0      0   
GCGAGAAAGCATTCCA-1_Flt3l        1    NaN      0      1      0      0      0   

                           11761T 11812G 13188T  ... 750G 7598A 827G 8292A  \
GCTGTTCAGTAACATG-1_Input        1      0      1  ...    1     1    1     1   
AGCCAGCGTCCGTGCA-1_Flt3l        1    NaN      1  ...  NaN     1    1     1   
TATTGCTTCTTGCGCT-1_Flt3l        1      0      1  ...    1     1    1     1   
CACTAAGTCCCACTAC-1_Control      1      0      1  ...    1     1    1     1   
GCGAGAAAGCATTCCA-1_Flt3l        1      0      1  ...    1     1    1     1   

                           8461T

,10397G,10463C,10559G,10589A,11251G,11453A,11719A,11761T,11812G,13188T,...,8292A,8461T,8697A,8743A,8860G,930A,9899C,15297C,15497A,5442C
GCTGTTCAGTAACATG-1_Input,1,0,0,1,0,0,0,1,0,1,...,1,1,0,0,1,0,0,0,0,0
AGCCAGCGTCCGTGCA-1_Flt3l,1,0,0,1,0,0,0,1,0,1,...,1,1,0,0,1,0,0,0,0,0
TATTGCTTCTTGCGCT-1_Flt3l,1,0,0,1,0,0,0,1,0,1,...,1,1,0,0,1,0,0,0,0,0
CACTAAGTCCCACTAC-1_Control,1,0,0,1,0,0,0,1,0,1,...,1,1,0,0,1,1,0,0,0,0
GCGAGAAAGCATTCCA-1_Flt3l,1,0,0,1,0,0,0,1,0,1,...,1,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATCCAGAAGTTCAGGG-1_Input,0,1,1,0,1,0,1,0,1,0,...,0,0,1,0,1,1,0,1,0,0
AGTCCGGTCACTGGTA-1_Input,0,1,1,0,1,0,1,0,1,0,...,0,0,1,0,1,1,1,0,0,0
CCAGATAGTGCAAGAC-1_Flt3l,0,0,1,0,1,0,1,0,1,0,...,0,0,1,0,1,1,1,0,0,0
AGTCCGGGTTCCCTTG-1_Control,0,1,1,0,1,0,1,0,1,0,...,0,0,1,0,1,1,1,0,0,1


In [7]:

bin_comb = pd.concat(list(bin_d.values()),axis=0).astype('Int32')#.fillna(0).astype(int)
bin_comb.to_csv(join(outdir, "combined_binary_na.csv"))
print("keep na as is")
bin_comb

keep na as is


,10397G,10463C,10559G,10589A,11251G,11453A,11719A,11761T,11812G,13188T,...,8292A,8461T,8697A,8743A,8860G,930A,9899C,15297C,15497A,5442C
GCTGTTCAGTAACATG-1_Input,1,0,0,1,0,0,0,1,0,1,...,1,1,0,0,1,0,0,<NA>,<NA>,<NA>
AGCCAGCGTCCGTGCA-1_Flt3l,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,1,<NA>,1,...,1,1,<NA>,<NA>,1,<NA>,<NA>,<NA>,<NA>,<NA>
TATTGCTTCTTGCGCT-1_Flt3l,1,0,0,1,0,0,0,1,0,1,...,1,1,0,0,1,0,0,<NA>,<NA>,<NA>
CACTAAGTCCCACTAC-1_Control,1,0,0,1,0,0,0,1,0,1,...,1,1,<NA>,0,1,1,0,<NA>,<NA>,<NA>
GCGAGAAAGCATTCCA-1_Flt3l,1,<NA>,0,1,0,0,0,1,0,1,...,1,1,0,0,1,<NA>,0,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATCCAGAAGTTCAGGG-1_Input,0,1,1,0,1,<NA>,1,0,1,0,...,0,0,1,<NA>,1,1,0,1,0,0
AGTCCGGTCACTGGTA-1_Input,0,1,1,0,1,<NA>,1,0,1,0,...,0,0,1,<NA>,1,1,1,0,0,0
CCAGATAGTGCAAGAC-1_Flt3l,<NA>,<NA>,1,<NA>,1,<NA>,1,0,1,0,...,0,0,1,<NA>,1,1,1,0,0,<NA>
AGTCCGGGTTCCCTTG-1_Control,0,1,1,0,1,<NA>,1,0,1,0,...,0,0,1,<NA>,1,1,1,0,0,1


In [8]:
curr_p.to_csv(join(outdir, "best_params.csv"))
